### A1



Wir verschaffen uns ein N x N Gitter, das zunächst nur mit Punkten besetzt ist

In [61]:
N = 10
grid = [['.']*n for _ in range(n)]

In [62]:
def show(grid):
    for row in grid:
        print(*row)

show(grid)

. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .


Auf die Elemente des grids können wir mit Koordinaten zugreifen.

In [63]:
grid[1][5] = 2     # Zeile 1, Spalte 3  
grid[6][9] = 2
show(grid)

. . . . . . . . . .
. . . . . 2 . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . 2
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .


Koordinaten, die wir nicht betreten dürfen, markieren wir mit '#'.

In [64]:
for i in range(3,N):
    grid[5][i] = '#'
show(grid)

. . . . . . . . . .
. . . . . 2 . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . # # # # # # #
. . . . . . . . . 2
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .


Mit der Breitensuche finden wir den kürzesten Weg zwischen den beiden Punkten.
Die Funktion nextPos ermittelt, was unsere möglichen Nachbarkoordinaten sind. 

In [65]:
dirs = [(1,0),(-1,0),(0,1),(0,-1)]   # die vier Bewegungsrichtungen
def nextPos(pos):
    tmp = []
    x, y = pos
    for xd,yd in dirs:
        x1, y1 = x+xd, y+yd         # Nachbarkoordinate
        if (0 <= x1 < N) and (0 <= y1 < N) and grid[x1][y1] != '#':   # im grid und betretbar?
            tmp.append((x1,y1))
    return tmp    
    

In [66]:
from collections import deque

def bfs(start):
    frontier =  deque([start])
    prev = {start:None}
    while frontier:
        pos = frontier.popleft()
        if pos == goal:
            return prev 
        for p in nextPos(pos):
            if p not in prev:
                frontier.append(p)
                prev[p] = pos
    return None 

def reconstructPath(prev,goal):
    pos = goal
    path = []
    while pos is not None:
        path.append(pos)
        pos = prev[pos]
    path.reverse()
    return path

In [67]:
start = (1,5)
goal = (6,9)
prev = bfs(start)
path = reconstructPath(prev,goal)
print(path)

[(1, 5), (2, 5), (3, 5), (4, 5), (4, 4), (4, 3), (4, 2), (5, 2), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9)]


Jetzt können wir den Pfad markieren

In [68]:
for (x,y) in path:
    grid[x][y] = 2
show(grid)

. . . . . . . . . .
. . . . . 2 . . . .
. . . . . 2 . . . .
. . . . . 2 . . . .
. . 2 2 2 2 . . . .
. . 2 # # # # # # #
. . 2 2 2 2 2 2 2 2
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
